# RSI 70-30 example

- An example strategy [for RSI rebalance](https://help.tokensets.com/en/articles/3541009-eth-btc-rsi-ratio-trading-set-details)

# Set up


In [1]:
import numpy as np

from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


# Load data

We use Binance data so we get a longer period of data.

In [2]:
import datetime
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.binance import create_binance_universe

strategy_universe = create_binance_universe(
    ["ETHUSDT", "BTCUSDT"],
    candle_time_bucket=TimeBucket.d1,
    stop_loss_time_bucket=TimeBucket.h4,
    start_at=datetime.datetime(2020, 1, 1),
    end_at=datetime.datetime(2024, 1, 1),
    include_lending=False
)


  0%|          | 0/2 [00:00<?, ?it/s]

# Show loaded trading universe

In [3]:
pairs = strategy_universe.data_universe.pairs  # Trading pairs metadata
candles = strategy_universe.data_universe.candles  # Candles for all trading pairs

print(f"We loaded {candles.get_candle_count():,} candles.")

for pair in pairs.iterate_pairs():
    print(pair)

We loaded 2,924 candles.
<Pair #1 ETH - USDT (0xe82ac67166a910f4092c23f781cd39e46582ec9c) at exchange binance>
<Pair #2 BTC - USDT (0x1d06ef1d6470d25f8e3d6f04f5acc111f176939c) at exchange binance>


# Trading algorithm

In [4]:
from tradingstrategy.chain import ChainId
from typing import List, Dict

from pandas_ta.overlap import ema
from pandas_ta.momentum import rsi
import pandas as pd

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.state.visualisation import PlotKind, PlotShape
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager

rsi_high = 70
rsi_low = 30

# List of pair descriptions we used to look up pair metadata
our_pairs = [
    (ChainId.centralised_exchange, "binance", "ETH", "USDT"),
    (ChainId.centralised_exchange, "binance", "BTC", "USDT"),
]
lookback_candles = 120


def decide_trades(
        timestamp: pd.Timestamp,
        strategy_universe: TradingStrategyUniverse,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:

    trades = []  # This is the list of trades we are going to  make in  this cycle
    cash = state.portfolio.get_current_cash()  # How much cash we have in a hand

    visualisation = state.visualisation  # Helper class to visualise strategy output
    current_rsi_values = {}
    current_price = {}

    # Calculate indicators
    for desc in our_pairs:  # Visualise RSI for all of our traded pairs
        pair = pairs.get_pair_by_human_description(desc)
        token = pair.base_token_symbol  # "ETH" or "BTC"
        pair_candles = candles.get_last_entries_by_pair_and_timestamp(pair, timestamp)
        assert pair_candles is not None
        rsi_series = rsi(pair_candles["close"], length=14)  # Will return None if the data buffer does not have enough days to look back

        current_rsi_values[pair] = None
        current_price[pair] = None

        if len(pair_candles) > 0:
            # We have enough data to get the latest price
            current_price[pair] = pair_candles["close"][-1]

        if rsi_series is not None:
            current_val = rsi_series[-1]
            if np.isfinite(current_val):
                # We have enough data and good value for RSI
                assert 0 < current_val < 100, f"RSI sanity check failed: {pair}: {current_val}"  # Check we are in expected range
                current_rsi_values[pair] = current_val

        # Draw RSI between its trigger zones
        if current_rsi_values[pair]:
            # Low (vertical line)
            visualisation.plot_indicator(
                timestamp,
                f"RSI {token} low trigger",
                PlotKind.technical_indicator_overlay_on_detached,
                rsi_low,
                detached_overlay_name="RSI {token}"
            )

            # High (vertical line)
            visualisation.plot_indicator(
                timestamp,
                f"RSI {token} high trigger",
                PlotKind.technical_indicator_overlay_on_detached,
                rsi_high,
                detached_overlay_name="RSI {token}"
            )

            # Current daily
            visualisation.plot_indicator(
                timestamp,
                f"RSI {token}",
                PlotKind.technical_indicator_overlay_on_detached,
                current_rsi_values[pair],
                detached_overlay_name="RSI {token}"
            )

    return trades

# Backtest

In [5]:
from tradeexecutor.strategy.cycle import CycleDuration

from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="RSI multipair",
    engine_version="0.3",
    decide_trades=decide_trades,
    client=client,
    cycle_duration=CycleDuration.cycle_1d,
    universe=strategy_universe,
    initial_deposit=10_000,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

  0%|          | 0/126230400 [00:00<?, ?it/s]

AssertionError: Expected USDC reserve, got USDT